# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Y. Wang  ->  Y. Wang  |  ['Y. Wang']
T. Kalda  ->  T. Kalda  |  ['T. Kalda']
J. Li  ->  J. Li  |  ['J. Li']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
R. Seeburger  ->  L. R. Seeburger  |  ['R. Seeburger']
J. Villasenor  ->  J. Villasenor  |  ['J. Villasenor']


Arxiv has 60 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.06309


extracting tarball to tmp_2504.06309...

 done.
Retrieving document from  https://arxiv.org/e-print/2504.06352


extracting tarball to tmp_2504.06352...

...

extracting tarball to tmp_2504.06968... done.
Retrieving document from  https://arxiv.org/e-print/2504.06973


extracting tarball to tmp_2504.06973...

 done.


J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
R. Seeburger  ->  L. R. Seeburger  |  ['R. Seeburger']
J. Villasenor  ->  J. Villasenor  |  ['J. Villasenor']


Found 190 bibliographic references in tmp_2504.06973/main.bbl.
Issues with the citations
syntax error in line 60: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.06352-b31b1b.svg)](https://arxiv.org/abs/2504.06352) | **Quadrupole signature as a kinematic diagnostic to constrain bar properties : implication for the Milky Way**  |
|| S. Ghosh, et al. -- incl., <mark>T. Kalda</mark> |
|*Appeared on*| *2025-04-10*|
|*Comments*| *15 pages, 1 table, 12 figures (including appendix), comments are welcome!*|
|**Abstract**|            The presence of a 'butterfly' or a quadrupole structure in the stellar mean radial velocity field of the Milky Way is well known from the Gaia and the APOGEE surveys. Past studies indicated that a stellar bar can excite such a quadrupole feature in the $<V_R>$ distribution. However, a systematic study investigating the co-evolution of bar and quadrupole structure is largely missing. Furthermore, whether this quadrupole structure in $<V_R>$ can be used as a robust kinematic diagnostic to constrain bar properties, particularly for the Milky Way, is still beyond our grasp. Here, we investigate the bar-induced quadrupole feature using a suite of isolated N-body models forming prominent bars and a sample of Milky Way-like barred galaxies from the TNG50 cosmological simulation. We demonstrate that the properties of the quadrupole (strength, length, and orientation) are strongly correlated with the bar properties, regardless of the choice of the stellar tracer population; thereby making the quadrupole feature an excellent kinematic diagnostic for constraining the bar properties. In presence of spirals, the estimator which takes into account the phase-angle of m=4 Fourier moment, serves as a more appropriate estimator for measuring the length of the quadrupole. Further, we constructed a novel Gaia-like mock dataset from a simulated bar model while incorporating the dust extinction and the broad trends of observational errors of the Gaia survey. The quadrupole properties (strength and length) estimated from those Gaia-like mock data are larger (~35-45 percent) when compared with their true values. We determined that the majority of this effect is due to the uncertainty in parallax measurement. This demonstrates the potential caveat of inferring Milky Way's bar properties by using the stellar kinematic information from the Gaia DR3 without properly accounting for the observational uncertainties.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.06973-b31b1b.svg)](https://arxiv.org/abs/2504.06973) | **HIP 15429: a newborn Be star on an eccentric binary orbit**  |
|| <mark>J. Müller-Horn</mark>, et al. -- incl., <mark>K. El-Badry</mark>, <mark>H.-W. Rix</mark>, <mark>R. Seeburger</mark>, <mark>J. Villasenor</mark> |
|*Appeared on*| *2025-04-10*|
|*Comments*| *submitted to Astronomy & Astrophysics*|
|**Abstract**|            We identified a new post-interaction binary, HIP 15429, consisting of a stripped star and a recently formed, rapidly rotating Be star companion ($v \sin i \approx 270$ km/s) sharing many similarities with recently identified bloated stripped stars. From orbital fitting of multi-epoch radial velocities we find a 221-day period. We also find an eccentricity of $e=0.52$, which is unexpectedly high as tides are expected to have circularised the orbit efficiently during the presumed recent mass transfer. The formation of a circumbinary disk during the mass transfer phase or the presence of an unseen tertiary companion might explain the orbit's high eccentricity. We determined physical parameters for both stars by fitting the spectra of the disentangled binary components and multi-band photometry. The stripped nature of the donor star is affirmed by its high luminosity at a low inferred mass ($\lesssim 1 \mathrm{M}_\odot$) and imprints of CNO-processed material in the surface abundances. The donor's relatively large radius and cool temperature ($T_{\mathrm{eff}} = 13.5 \pm 0.5$ kK) suggest that it has only recently ceased mass transfer. Evolutionary models assuming a 5-6 $\mathrm{M}_\odot$ progenitor can reproduce these parameters and imply that the binary is currently evolving towards a stage where the donor becomes a subdwarf orbiting a Be star. The remarkably high eccentricity of HIP 15429 challenges standard tidal evolution models, suggesting either inefficient tidal dissipation or external influences, such as a tertiary companion or circumbinary disk. This underscores the need to identify and characterise more post-mass transfer binaries to benchmark and refine theoretical models of binary evolution.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.06309-b31b1b.svg)](https://arxiv.org/abs/2504.06309) | **AI-Driven Reconstruction of Large-Scale Structure from Combined Photometric and Spectroscopic Surveys**  |
|| W. Du, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-04-10*|
|*Comments*| **|
|**Abstract**|            Galaxy surveys are crucial for studying large-scale structure (LSS) and cosmology, yet they face limitations--imaging surveys provide extensive sky coverage but suffer from photo-$z$ uncertainties, while spectroscopic surveys yield precise redshifts but are sample-limited. To take advantage of both photo-$z$ and spec-$z$ data while eliminating photo-$z$ errors, we propose a deep learning framework based on a dual UNet architecture that integrates these two datasets at the field level to reconstruct the 3D photo-$z$ density field. We train the network on mock samples representative of stage-IV spectroscopic surveys, utilizing CosmicGrowth simulations with a $z=0.59$ snapshot containing $2048^3$ particles in a $(1200~h^{-1}\rm Mpc)^3$ volume. Several metrics, including correlation coefficient, MAE, MSE, PSNR, and SSIM, validate the model's accuracy. Moreover, the reconstructed power spectrum closely matches the ground truth at small scales ($k \gtrsim 0.06~h/\rm Mpc$) within the $1\sigma$ confidence level, while the UNet model significantly improves the estimation of photo-$z$ power spectrum multipoles. This study demonstrates the potential of deep learning to enhance LSS reconstruction by using both spectroscopic and photometric data.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.06415-b31b1b.svg)](https://arxiv.org/abs/2504.06415) | **Galactic-scale Quasar Pairs from the Sloan Digital Sky Survey and Gaia DR3**  |
|| Y. Jiang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-04-10*|
|*Comments*| *17 pages, 9 figures, 3 tables*|
|**Abstract**|            We preform a systematic search for galactic-scale quasar pairs and small-scale (<3 arcsec) lenses using the SDSS DR16 quasar catalog and Gaia DR3. Candidate double quasars (both are unobscured) are identified as Gaia resolved pairs around spectroscopically confirmed SDSS quasars ($L_{\rm bol} > 10^{44.5} \ {\rm erg \ s^{-1}}$) at $0.5 < z \lesssim 4.5$. Gaia astrometric information and SDSS spectral decomposition are used to exclude foreground star superpositions, which dominate ($\gtrsim 80\%$ of) the pair sample. We identify $136$ double quasar candidates from $1120$ Gaia-resolved pairs after a magnitude and redshift cut of $G<20.5$ and $z>0.5$ (803 double quasars out of 2,497 pairs without any cuts applied) with separations of $\sim$ 0.3 - 3 arcsec, corresponding to projected physical separations of $\sim$ 3 - 30 kpc at the median redshift of the sample of $z = 1.7$. We estimate an overall double quasar (lens and physical pairs combined) fraction using this sample, corrected for pair-resolving completeness, of $5.7_{-0.3}^{+0.3} \times 10^{-4}$ (bootstrapping errors). This double quasar fraction increases toward smaller separations, consistent with earlier findings. We also find little redshift evolution of the double quasar fraction for the luminous SDSS sample, consistent with previous observations and simulation predictions. However, the observed fraction is lower than simulation predictions by $\sim$0.8 - 1.6 dex, suggesting a significant population of obscured quasar pairs are missed in our search. Future wide-area space missions targeting both unobscured and obscured quasar pairs at sub-arcsec resolutions will reveal this population of obscured quasar pairs, and extend to much lower AGN luminosities.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.06616-b31b1b.svg)](https://arxiv.org/abs/2504.06616) | **Search for Type IL Gamma-ray Bursts: Criterion, Results, Verification and Physical Implication**  |
|| W.-J. Tan, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-04-10*|
|*Comments*| *21 pages, 10 figures, submitted to ApJS*|
|**Abstract**|            As an interesting subclass of gamma-ray burst (GRB), Type IL GRB (such as GRB 211211A and GRB 230307A) features a long-duration prompt emission but originating from compact binary merger. The "long duration" emisison of Type IL GRB are dominately composed of the main burst, rather than the extended emission, differentiating them from the traditional "long-short" GRB (e.g., GRB 060614). Previous study has reported several Type IL GRBs by visual inspection of their light curves. In this work, we established a detailed criterion to identify Type IL GRBs by light curve, and then systematically searched the archival \textit{Fermi}/GBM data with this criterion, resulting in a sample of 5 type IL GRBs from January 1, 2014 to January 1, 2024, i.e. GRB 230307A, GRB 211211A, GRB 200914A, GRB 200311A and GRB 170228A. Apart from the light curve pattern, we find that the temporal and spectral properties of these 5 GRBs also support this classification. Interestingly, we find that the energy ratio between extended emission and main emission is almost constant ($\sim0.7$, with small scattering) for these GRBs, which have strong implication on the mechanism of Type IL burst. We discuss theoretical models to interpret the progenitor, central engine, and extended emission of these Type IL bursts.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.06968-b31b1b.svg)](https://arxiv.org/abs/2504.06968) | **Probable evidence for a transient mega-electron volt emission line in the GRB 221023A**  |
|| L.-Y. Jiang, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-04-10*|
|*Comments*| *20 pages, 5 figures, 3 tables. Publication in the Nature Communications*|
|**Abstract**|            Detection of spectral line in gamma-ray bursts (GRBs) is importance for studying GRB physics, as it provides insights into the composition and physical conditions of the GRB environment. However, progress in detecting X-ray or gamma-ray emission and absorption lines in GRB spectra has been relatively slow, only the narrow emission line feature of about 10 MeV found in GRB 221009A has exhibited a significance exceeding $5 \sigma$. Here, we report the probable evidence of a narrow emission feature at about 2.1 mega-electron volts (MeV) in the spectrum of GRB 221023A. The highest statistical significance of this feature is observed in the time interval between 8 and 30 seconds after Fermi Gamma-Ray Burst Monitor trigger, with the chance probability value $<2.56 \times 10^{-5}$ (after accounting for the look-elsewhere effect), corresponding to a Gaussian-equivalent significance $> 4.20 \sigma$. We interpret this feature as being generated through the de-excitation of excited electrons in the relativistic hydrogen-like high-atomic-number ions entrained in the GRB jet.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.06352/./new0300_vR_vs_x_y_error_comparison_thorough.png', 'tmp_2504.06352/./bar_vs_quadrupoleRelations_correlation_TracerDependence_for_paper.png', 'tmp_2504.06352/./bar_Quadrupole_strength_2lobevs4lobe_rthcikE00_t360.png']
copying  tmp_2504.06352/./new0300_vR_vs_x_y_error_comparison_thorough.png to _build/html/
copying  tmp_2504.06352/./bar_vs_quadrupoleRelations_correlation_TracerDependence_for_paper.png to _build/html/
copying  tmp_2504.06352/./bar_Quadrupole_strength_2lobevs4lobe_rthcikE00_t360.png to _build/html/
exported in  _build/html/2504.06352.md
    + _build/html/tmp_2504.06352/./new0300_vR_vs_x_y_error_comparison_thorough.png
    + _build/html/tmp_2504.06352/./bar_vs_quadrupoleRelations_correlation_TracerDependence_for_paper.png
    + _build/html/tmp_2504.06352/./bar_Quadrupole_strength_2lobevs4lobe_rthcikE00_t360.png
found figures ['tmp_2504.06973/./figures/spectral_fit_stripped_star.png', 'tmp_2504.06973/./figures/Halpha_quadrature.png', 'tmp_2504

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{\mbox{\>{\rm km  s^{-1}}}}$
$\newcommand{\pc}{\>{\rm pc}}$
$\newcommand{\kpc}{\mbox{\>{\rm kpc}}}$
$\newcommand{\kmsk}{\mbox{\>{\rm km  s^{-1}  kpc^{-1}}}}$
$\newcommand{\Gyr}{\mbox{\>{\rm Gyr}}}$
$\newcommand{\Myr}{\mbox{\>{\rm Myr}}}$
$\newcommand{\yr}{\mbox{\>{\rm yr}}}$
$\newcommand{\Msun}{\>{\rm M_{\odot}}}$
$\newcommand{\Rd}{\mbox{R_{\rm d}}}$
$\newcommand{\zd}{\mbox{z_{\rm d}}}$
$\newcommand{\md}{\mbox{M_{\rm d}}}$
$\newcommand$
$\newcommand{\avg}[1]{\mbox{\left<{#1}\right>}}$
$\newcommand{\sig}[1]{\mbox{\sigma_{#1}}}$
$\newcommand{\feh}{\mbox{\rm[Fe/H]}}$
$\newcommand{\al}{\mbox{\rm \alpha}}$
$\newcommand{\alfe}{\mbox{\rm[O/Fe]}}$
$\newcommand{\tf}{\mbox{\tau}}$
$\newcommand{\rs}{\mbox{R_s}}$
$\newcommand{\abar}{\mbox{A_{\rm bar}}}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Quadrupole signature as a kinematic diagnostic to constrain bar properties : implication for the Milky Way

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.06352-b31b1b.svg)](https://arxiv.org/abs/2504.06352)<mark>Appeared on: 2025-04-10</mark> -  _15 pages, 1 table, 12 figures (including appendix), comments are welcome!_

</div>
<div id="authors">

S. Ghosh, et al. -- incl., <mark>T. Kalda</mark>

</div>
<div id="abstract">

**Abstract:** The presence of a `butterfly' or a quadrupole structure in the stellar mean radial velocity ( $\avg{V_R}$ ) field of the Milky Way is well known from the $\gaia$ and the APOGEE surveys. Past studies indicated that a stellar bar can excite such a quadrupole feature in the $\avg{V_R}$ distribution. However, a systematic study investigating the co-evolution of bar and quadrupole structure is largely missing. Furthermore, whether this quadrupole structure in $\avg{V_R}$ can be used as a robust kinematic diagnostic to constrain bar properties, particularly for the Milky Way, is still beyond our grasp. Here, we investigate the bar-induced quadrupole feature using a suite of isolated $N$ -body models forming prominent bars and a sample of Milky Way-like barred galaxies from the TNG50 cosmological simulation.  We demonstrate that the properties of the quadrupole (strength, length, and orientation) are strongly correlated with the bar properties, regardless of the choice of the stellar tracer population; thereby making the quadrupole feature an excellent kinematic diagnostic for constraining the bar properties. In presence of spirals, the estimator which takes into account the phase-angle of $m = 4$ Fourier moment, serves as a more appropriate estimator for measuring the length of the quadrupole.  Further, we constructed a novel $\gaia$ -like mock dataset from a simulated bar model while incorporating the dust extinction and the broad trends of observational errors of the $\gaia$ survey. The quadrupole properties (strength and length) estimated from those $\gaia$ -like mock data are larger ( $\sim 35-45$ percent) when compared with their true values. We determined that the majority of this effect is due to the uncertainty in parallax measurement. This demonstrates the potential caveat of inferring Milky Way's bar properties by using the stellar kinematic information from the $\gaia$ DR3 without properly accounting for the observational uncertainties.

</div>

<div id="div_fig1">

<img src="tmp_2504.06352/./new0300_vR_vs_x_y_error_comparison_thorough.png" alt="Fig9" width="100%"/>

**Figure 9. -** _Investigating the effects of different uncertainties:_ Face-on distribution of $\avg{v_R}$ for the \gaia-like mock dataset at $t=3\mathrm{ Gyr}$, showcasing how the observed quadrupole feature is affected by errors in proper motion, radial velocity ($V_R$), and parallax. The _top left panel_ is for stars in the snapshot (no uncertainties and no magnitude cut) while _top middle panel_ shows the magnitude limited $G \leq 15.5$ mock dataset with no errors. The _top right panel_ and the _bottom left panel_ show the mock datasets with only proper motion (PM) uncertainty and both the PM and radial velocity uncertainties, respectively. The _bottom middle panel_ shows the mock dataset resampled according to the errors in the \gaia DR3 RGB subsample (see section \ref{sec:gaia_like_testing}), and the _bottom right panel_ shows the mock dataset where all the particles have had their $V_R$ set to zero prior to applying observational effects. For further details, see sect. \ref{sec:application_to_MW}. (*fig:0300_parallax_contamination_analysis*)

</div>
<div id="div_fig2">

<img src="tmp_2504.06352/./bar_vs_quadrupoleRelations_correlation_TracerDependence_for_paper.png" alt="Fig5" width="100%"/>

**Figure 5. -** _Dependence on the stellar tracer population:_ correlation between the bar strength, $S_{\rm bar}$ and the strength of the quadrupole, $S_{\rm quadrupole}$(_top row_), and correlation between the bar length, $R_{\rm bar}$ and the length of the quadrupole, $R_{\rm quadrupole}$(_bottom row_), computed using thin disc particles (_left panels_) and thick disc particles (_right panels_), for all thin+thick models and the sim6 model (see the legend). The black dash line denotes the best-fit straight line (of the form $Y = AX+B$) while the grey shaded region denotes the 5-$\sigma$ scatter around the best-fit line. Regardless of the stellar tracer population, the length and strength of the quadrupole remain strongly correlated with the length and the strength of the bar. (*fig:bar_quadrupole_correlation_Tracer_dependence*)

</div>
<div id="div_fig3">

<img src="tmp_2504.06352/./bar_Quadrupole_strength_2lobevs4lobe_rthcikE00_t360.png" alt="Fig1" width="100%"/>

**Figure 1. -** _Top panel:_ Radial variation of the amplitude of the $m=4$ Fourier moment of the mean radial velocity, $\avg{V_R}$, calculated at $t = 9 $\Gyr$$ for the model rthick0.0, while putting the bar at different orientations (see the legend). _Bottom panel:_ radial variation of the corresponding phase-angle ($\varphi_4$) of $m=4$ Fourier moment of the mean radial velocity, $\avg{V_R}$. The points denoting the radial variation computed from the mean radial velocity, $\avg{V_R}$ while the horizontal lines denote the corresponding _true_ bar orientation. `Half-extent' refers to the scenario where stars falling only in the positive half ($x \geq 0$) are considered while computing the Fourier moments. (*fig:bar_quadrupole_orientation_quantification*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.06352"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\aj}{Astronomical Journal}$
$\newcommand{\apj}{Astrophysical Journal}$
$\newcommand{\aap}{Astronomy \& Astrophysics}$
$\newcommand{\jcap}{Journal of Cosmology and Astroparticle Physics}$
$\newcommand{\aapr}{The Astronomy and Astrophysics Review}$
$\newcommand{\mnras}{Monthly Notices of the Royal Astronomical Society}$
$\newcommand{\araa}{Annual Review of Astronomy and Astrophysics}$
$\newcommand{\Msun}{\mathrm{M}_\odot}$</div>



<div id="title">

# HIP 15429: a newborn Be star on an eccentric binary orbit

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.06973-b31b1b.svg)](https://arxiv.org/abs/2504.06973)<mark>Appeared on: 2025-04-10</mark> -  _submitted to Astronomy & Astrophysics_

</div>
<div id="authors">

<mark>J. Müller-Horn</mark>, et al. -- incl., <mark>K. El-Badry</mark>, <mark>H.-W. Rix</mark>, <mark>R. Seeburger</mark>, <mark>J. Villasenor</mark>

</div>
<div id="abstract">

**Abstract:** Interaction in close binary systems is common in massive stars. The mass donor is stripped of its hydrogen envelope and evolves to become a hot helium star, whereas the accretor gains mass and angular momentum, spinning up in the process. However, the small number of well-constrained post-interaction binary systems currently limits detailed comparisons with binary evolution models.    We identified a new post-interaction binary, HIP 15429, consisting of a stripped star and a recently formed, rapidly rotating Be star companion ( $v_\mathrm{rot} \sin i \approx 270 $ km/s) sharing many similarities with recently identified bloated stripped stars.    From orbital fitting of multi-epoch radial velocities we find a 221-day period. We also find an eccentricity of $e=0.52$ , which is unexpectedly high as tides are expected to have circularised the orbit efficiently during the presumed recent mass transfer. The formation of a circumbinary disk during the mass transfer phase or the presence of an unseen tertiary companion might explain the orbit's high eccentricity.    We determined physical parameters for both stars by fitting the spectra of the disentangled binary components and multi-band photometry. The stripped nature of the donor star is affirmed by its high luminosity at a low inferred mass ( $\lesssim 1 \mathrm{M}_\odot$ ) and imprints of CNO-processed material in the surface abundances. The donor's relatively large radius and cool temperature ( $T_{\rm eff} = 13.5 \pm 0.5 $ kK) suggest that it has only recently ceased mass transfer.    Evolutionary models assuming a 5-6 $ \mathrm{M}_\odot$ progenitor can reproduce these parameters and imply that the binary is currently evolving towards a stage where the donor becomes a subdwarf orbiting a Be star.    The remarkably high eccentricity of HIP 15429 challenges standard tidal evolution models, suggesting either inefficient tidal dissipation or external influences, such as a tertiary companion or circumbinary disk. This underscores the need to identify and characterise more post-mass transfer binaries to benchmark and refine theoretical models of binary evolution.

</div>

<div id="div_fig1">

<img src="tmp_2504.06973/./figures/spectral_fit_stripped_star.png" alt="Fig22" width="100%"/>

**Figure 22. -** Spectral constraints on the stellar parameters of the narrow-lined stripped star. The panels show the disentangled spectrum (black) compared to PoWR models with the best-fit parameters ($T_\mathrm{eff} = 13.5,\text{kK}$, $\log g = 2.25$). The models differ in composition: one assumes solar hydrogen, helium and metal abundances \citep{Asplund+2021}, while our fiducial model (blue) is He- and N-enriched, characteristic of a stripped star. The narrow H-Balmer absorption lines support a low surface gravity, while the stripped-star model provides a better fit to the He and N absorption features. Metal lines not present in the model spectra (e.g., Si II 4153, 4163 Å) were not included in the atomic line lists. (*fig:stripped_star_models*)

</div>
<div id="div_fig2">

<img src="tmp_2504.06973/./figures/Halpha_quadrature.png" alt="Fig1.1" width="50%"/><img src="tmp_2504.06973/./figures/Halpha_shortterm_variability.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** _Top:_{Balmer emission lines at quadrature.} Two TRES spectra taken close to quadrature (red and blue lines) and one spectrum close to the system's barycentric velocity (grey line). The grey dotted lines indicate the rest-frame wavelengths of the Balmer H$_\alpha$ and carbon C II 6572 lines. Orbital motion of the narrow-lined star is apparent from wavelength shifts in the C II lines, whereas the H$_\alpha$ line shows variable emission line profiles.\ _Bottom:_{Short-term variability of emission line profiles.} Three TRES spectra observed at $t = \mathrm{MJD}   60235$, $t + 5 $d and $t+ 9 $d, that is, spanning $<5\%$ of the orbital period. Solid grey, red and blue lines show the variable H$_\alpha$ emission line profiles. The rest-frame wavelength of H$_\alpha$ is shown as in the top panel. Narrow lines at 6574, 6577 Å   originate from variable telluric absorption. (*fig:spectral_variability*)

</div>
<div id="div_fig3">

<img src="tmp_2504.06973/./figures/ecc_literature_comparison.png" alt="Fig25" width="100%"/>

**Figure 25. -** {Period-eccentricity diagram for known post-mass transfer binaries.} Recently discovered bloated stripped star binaries with O/Be companions \citep{Shenar+2020,ElBadry_Quataert2021,Villasenor+2023,Ramachandran+2024,Pauli+2022} are indicated with red diamond markers. These include the parameter values for HIP 15429 determined in this work (Müller-Horn et al. 2025). Be + subdwarf binaries with known orbital parameters are shown with light blue triangles \citep{Mourard+2015,Chojnowski+2018,Peters+2008,Peters+2013,Klement+2022,Klement+2024,Wang+2023}. We plot the short-period subdwarf binaries compiled by \cite{Edelmann+2005} and \cite{Kupfer+2015} and long-period ($>500 $d orbits) subdwarf binaries discovered by \cite{Deca+2012,Barlow+2013,Vos+2012,Vos+2013,Vos+2017} with dark blue triangles. Likely post-common envelope binaries with white dwarf or main sequence stars analysed by \cite{Yamaguchi+2024} are shown with yellow stars. (*fig:period_eccentricity_comparison*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.06973"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

434  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

15  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
